In [26]:
#jai bajrangbali
#creation of models

In [27]:
import tensorflow as tf

# Check if TensorFlow is detecting the GPU
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

# Check detailed information about the GPU
if len(physical_devices) > 0:
    print(tf.config.get_logical_device_configuration(physical_devices[0]))
else:
    print("No GPU available.")
!nvidia-smi


Num GPUs Available:  0
No GPU available.
/bin/bash: line 1: nvidia-smi: command not found


In [28]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from gym import Env
from gym.spaces import Discrete, Box
import random
from collections import deque

import gc
import pickle


In [29]:
# cretion of model
q_model = Sequential([
   Dense(128,activation='relu'),
   Dense(512,activation='relu'),
   Dense(512,activation='relu'),
   Dense(1024,activation='relu'),
   Dense(1024,activation='relu'),
   Dense(841,activation='linear')
])

q_model_target = Sequential([
   Dense(128,activation='relu'),
   Dense(512,activation='relu'),
   Dense(512,activation='relu'),
   Dense(1024,activation='relu'),
   Dense(1024,activation='relu'),
   Dense(841,activation='linear')
])
q_model.compile(optimizer='adam',loss='mse')
q_model_target.compile(optimizer='adam',loss='mse')
#q_model_target.set_weights(q_model.get_weights())  # Synchronize weights

In [30]:
# Matrix for room structure (29*29)=5 rooms per floor and 5 floors with 4 stairs
matrix = [
  # [1,2,3,4,5,S,1,2,3,4,5,S,1,2,3,4,5,S,1,2,3,4,5,S,1,2,3,4,5]
    [0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room1
    [1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room2
    [1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room3
    [1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room4
    [0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room5
    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #STAIR 1

    [0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room1
    [0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room2
    [0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room3
    [0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room4
    [0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room5
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0], #STAIR 2

    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room1
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room2
    [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0], #Room3
    [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0], #Room4
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room5
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0], #STAIR 3

    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0], #Room1
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0], #Room2
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0], #Room3
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0], #Room4
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0], #Room5
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1], #STAIR 4

    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0], #Room1
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0], #Room2
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1], #Room3
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1], #Room4
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #Room5
]

In [31]:
class PathEnv(Env):
    def __init__(self):

        self.action_space = Discrete(841)
        self.n=29

        self.state = random.randint(0,29)
        self.link_matrix = np.array(matrix)
        self.exit_rooms = np.array([28,])



    def step(self, action):
        vi = action%(self.n)
        vj = action//(self.n)
        if(self.state in self.exit_rooms):
            return self.state,0,True
        if(vi!=self.state): #<-----------------------
          return self.state,-10,False

        if(self.link_matrix[vi][vj] == 1 and vj in self.exit_rooms):
          self.state=vj
          return vj,1,True
        else:
          if(self.link_matrix[vi][vj]==0):
            return self.state,-10,False
          else:
            self.state = vj
            return vj,-1,False

    def render(self):
        return self.state

    def reset(self):
        # Reset shower temperature
        self.state = random.randint(0,29)#changed 3 to 29
        return self.state


In [32]:
class FireEnv(Env):
    def __init__(self):
        self.action_space = Discrete(841)
        self.n=29
        self.bottleneck_for_room = [8,9,8,5,9,0,8,9,8,5,9,0,8,9,8,5,9,0,8,9,8,5,9,0,8,9,8,5,0]
        self.observation_space = Box(low=0,high=10,shape=(29,))
        self.link_matrix = np.array(matrix)
        self.p_S = np.array([8,9,8,5,9,0,8,9,8,5,9,0,8,9,8,5,9,0,8,9,8,5,9,0,8,9,8,5,0])
        self.S = self.p_S
        self.state = random.randint(0,29)  #new line added
        # self.S = np.random.randint(10,size=5)
        # self.S[4]=0
        self.exit_rooms = np.array([28,])
        self.p_fire_in_room = np.array([0.1,0.2,2,0.5,0.1,0,0.1,0.2,2,0.5,0.1,0,0.1,0.2,2,0.5,0.1,0,0.1,0.2,2,0.5,0.1,0,0.1,0.2,2,0.5,0])
        self.fire_in_room = np.array([1,19,200,12,1,0.1,6,21,54,12,2,0.3,1,19,189,12,6,0,1,25,241,6,2,1,1,19,200,12,0])
        self.p_t=1
        self.t=1
        self.t = 0
        self.p = 0



    def step(self, action):
        vi = action%(self.n)
        vj = action//(self.n)
        # print("vi=",vi," vj=",vj)

        # self.t=self.t+1

        if np.sum(self.S)==0:
            #return self.S,0,True
            return 28,0,True
        # self.fire_in_room = self.fire_in_room + np.random.rand(self.n)/10
        self.fire_in_room = self.fire_in_room + np.array([0.005,0.02,0.04,0.006,0.02,0,0.005,0.02,0.04,0.006,0.02,0,0.005,0.02,0.04,0.006,0.02,0,0.005,0.02,0.04,0.006,0.02,0,0.005,0.02,0.04,0.006,0])

        if self.p >= random.random():
          #return self.S,-1*pow((self.fire_in_room[vj]),self.t),False
          return vj,-1*pow((self.fire_in_room[vj]),self.t),False
        else:
          if np.sum(self.S) == 0:
            return 28,0,True
          elif self.S[vi]==0:
            return self.state,-(1.2)*(pow(np.max(self.fire_in_room),self.t)),False
          elif (self.link_matrix[vi][vj]!=0 and vj in self.exit_rooms):
            self.S[vi]=self.S[vi]-1
            if(np.sum(self.S)==0):
                return self.state,+100/(self.link_matrix[vi][vj]),True
            return self.state,+100/(self.link_matrix[vi][vj]),False
          elif self.link_matrix[vi][vj]==0:
            return vi,-2*(pow(np.max(self.fire_in_room),self.t)),False
          elif self.S[vj] == self.bottleneck_for_room[vj]:
            return vi,-(0.5)*(pow(np.max(self.fire_in_room),self.t))*self.link_matrix[vi][vj],False
          else:
            self.S[vi]=self.S[vi]-1
            self.S[vj]=self.S[vj]+1
            return vj,-1*(pow(self.fire_in_room[vj],self.t))*self.link_matrix[vi][vj],False



    def empty_state(self):
        arr = np.zeros(self.n)
        # k = random.randint(0,n-1)
        # while k in self.exit_rooms:
        #     k=random.randint(0,n-1)
        # arr[k]=random.randint(1,self.bottleneck_for_room)
        return arr

    def render(self):
        return self.state

    def reset(self):
        # self.S = self.p_S
        # self.S = np.random.randint(10,size=5)
        # self.S=self.p_S
        #array1 = np.full(29, 5)

        self.S= np.full(29, 5)
        self.S[28]=0
        # self.t = self.p_t
        self.t = 1
        # self.fire_in_room = self.p_fire_in_room
        # self.fire_in_room = np.array([0.1,0.2,2,0.5,0])
        self.fire_in_room = np.array([1,1.3,8,2,0.1,0,1,1.3,8,2,0.1,0,1,1.3,8,2,0.1,0,1,1.3,8,2,0.1,0,1,1.3,8,2,0])
        # self.S[4]=0
        return self.state

In [33]:
Env=PathEnv()
fire_states=np.zeros(29)
state=Env.reset()
print(type(state))
state=float(state)
print(type(state))
state=np.hstack((state,fire_states))
print(state.shape)
state=np.reshape(state,(1,30))
print(state.shape)
#shape of state is 1,26

<class 'int'>
<class 'float'>
(30,)
(1, 30)


In [34]:
#chosing of new action in int datatype

def chose_action(state,epsilon):
  if random.random()>=epsilon:
    state1=np.reshape(state,(1,30))
    modelvalues=q_model.predict(state1)
    action=modelvalues.argmax()
    action=int(action)
  else:
    action=random.randint(0,840)
  return action

In [35]:

print(type(state))
state=state.astype(np.float32)
print(state.shape)
q_model.predict(state)
def soft_update(main_model, target_model, tau):

    print("it is soft update")
    for target_var, main_var in zip(target_model.variables, main_model.variables):
        target_var.assign(tau * main_var + (1 - tau) * target_var)

<class 'numpy.ndarray'>
(1, 30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step


In [36]:
memory = deque(maxlen=50000)
#print(q_model_target.output_shape)
print(q_model.input_dtype)
def store_experience(memory, state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

float32


In [37]:

def training(memory,epsilon,gamma):
    total_target=np.zeros((10000,841))
    next_states=np.zeros((10000,30))
    states=np.zeros((10000,30))
    minibatch = random.sample(memory, 10000)
    _, actions, rewards, next_states, dones = zip(*minibatch)
    next_states=np.array(next_states)
    total_target = q_model_target.predict(next_states)
    total_target=np.squeeze(total_target,axis=1)
    
    print(total_target.shape)
    
    i=0
    for state, action, reward, next_state, done in minibatch:
        states[i]=state
        if done:
            print(epsilon)
            total_target[i][action] = reward
            i=i+1
            epsilon=epsilon-0.1
    
        else:
            #next_state=np.hstack(next_state,fire_states)
            total_target[i][action] = reward + gamma * np.amax(total_target[i])

            i=i+1
            
    print(states)            
    q_model.fit(states, total_target, epochs=1, verbose=0)
    soft_update(q_model, q_model_target, tau)


In [ ]:
episodes=5
gamma=0.995
tau=0.4
epsilon=0.1
import numpy as np
store=np.zeros(episodes)
for episode in range(episodes):
  state=PathEnv.reset(self=Env)
  state=float(state)
  state=np.hstack((state,fire_states))
  state=np.reshape(state,(1,30))
  done=False
  i=0
  print(episode)
  while done==False and i<=10000:
    i=i+1
    #print(i)
    action=chose_action(state,epsilon)
    next_state,reward,done=Env.step(action)
    next_state=np.array(float(next_state))
    next_state=np.hstack((next_state,fire_states))
    next_state=np.reshape(next_state,(1,30))
    store_experience(memory, state, action, reward, next_state, done)#state is array of 1,30 and next state is int32
    state=next_state
  print(store)  
  store[episode]=i
  training(memory,epsilon,gamma)1

q_model.save("q_model.keras")

In [1]:
import numpy as np
import tensorflow as tf
from collections import deque
import random

# Set parameters for the environment
state_size = 10  # Example state dimension
action_size = 4  # Example action space size
learning_rate = 0.001
gamma = 0.99  # Discount factor

# Initialize Q and Target models
def build_model(state_size, action_size, learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, input_dim=state_size, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(action_size, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return model

q_model = build_model(state_size, action_size, learning_rate)
q_model_target = build_model(state_size, action_size, learning_rate)
q_model_target.set_weights(q_model.get_weights())  # Synchronize weights

# Replay memory
memory = deque(maxlen=2000)

# Epsilon-greedy strategy
epsilon = 1.0  # Initial exploration rate
epsilon_min = 0.01
epsilon_decay = 0.995

# Action selection
def select_action(state, epsilon, q_model):
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)  # Explore
    q_values = q_model.predict(state)
    return np.argmax(q_values[0])  # Exploit

# Store experience
def store_experience(memory, state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

# Train the Q-network
def train_q_network(q_model, q_model_target, memory, batch_size):
    if len(memory) < batch_size:
        return

    minibatch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
        target = q_model.predict(state)
        if done:
            target[0][action] = reward
        else:
            t = q_model_target.predict(next_state)
            target[0][action] = reward + gamma * np.amax(t[0])

        q_model.fit(state, target, epochs=1, verbose=0)

# Update target network
def update_target_network(q_model, q_model_target):
    q_model_target.set_weights(q_model.get_weights())

# Simulate custom environment
def custom_environment_step(state, action):
    # Placeholder for environment logic
    next_state = np.random.rand(state_size)  # Example state transition
    reward = np.random.rand()  # Example reward
    done = random.choice([True, False])  # Random termination
    return next_state, reward, done

# Training loop
episodes = 1000
batch_size = 32

for episode in range(episodes):
    state = np.random.rand(1, state_size)  # Example initial state
    total_reward = 0
    done = False

    while not done:
        action = select_action(state, epsilon, q_model)
        next_state, reward, done = custom_environment_step(state, action)
        store_experience(memory, state, action, reward, np.reshape(next_state, [1, state_size]), done)
        train_q_network(q_model, q_model_target, memory, batch_size)
        state = np.reshape(next_state, [1, state_size])
        total_reward += reward

    update_target_network(q_model, q_model_target)

    # Decay epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f"Episode {episode + 1}, Total Reward: {total_reward:.2f}, Epsilon: {epsilon:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode 1, Total Reward: 0.26, Epsilon: 0.99
Episode 2, Total Reward: 0.47, Epsilon: 0.99
Episode 3, Total Reward: 0.60, Epsilon: 0.99
Episode 4, Total Reward: 0.86, Epsilon: 0.98
Episode 5, Total Reward: 0.77, Epsilon: 0.98
Episode 6, Total Reward: 1.08, Epsilon: 0.97
Episode 7, Total Reward: 3.05, Epsilon: 0.97
Episode 8, Total Reward: 0.63, Epsilon: 0.96
Episode 9, Total Reward: 1.50, Epsilon: 0.96
Episode 10, Total Reward: 0.51, Epsilon: 0.95
Episode 11, Total Reward: 1.50, Epsilon: 0.95
Episode 12, Total Reward: 0.67, Epsilon: 0.94
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Episode 13, Total Reward: 2.06, Epsilon: 0.94
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Episode 14, Total Reward: 0.97, Epsilon: 0.93
Episode 15, Total Reward: 0.13, Epsilon: 0.93
Episode 16, Total Reward: 0.78, Epsilon: 0.92
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


KeyboardInterrupt: 

In [ ]:
"#creatin of dataset to train
m=25#states of fire environment
mt=10000
n=1
fire_states=np.zeros(m)
dones=np.zeros(mt)
states=np.zeros((mt,m+n))
rewards=np.zeros(mt)
next_states=np.zeros((mt,m+n))
actions=np.zeros(mt)
s=np.array(Env.render())
for i in range(mt):
  Env.reset()
  s=np.array(Env.render())

  state=np.hstack((s,fire_states))
  states[i]=state
  action=chose_action(state,1)
  s,reward,done=Env.step(action)
  next_states[i]=np.hstack((s,fire_states))
  rewards[i]=reward
  actions[i]=action
 # print(state)
  dones[i]=done
qt_values=q_model_target.predict(next_states)
qtmax_values=np.amax(qt_values,axis=1)
target_values=rewards+qtmax_values
q_values = q_model.predict(states)
for i, action in enumerate(actions):
  q_values[i, action] = target_values[i]
print(qt_values.shape)

q_model.fit(states,q_vales)


In [ ]:
q_value=q_model.predict(states)

In [ ]:
a=q_value.argmax(axis=1)
print(a.shape)
print(a)
print(rewards.shape)
print(states)

In [ ]:
#CODE TO CHECK THE MODEL
s=np.array(Env.render())
done=False
steps=0
while done==False:
  steps=steps+1

  state=np.hstack((s,fire_states))
  action=chose_action(state,0.8)
  s,reward,done=Env.step(action)
  reward_s=rewards+reward
  print(steps,reward,s)
 #state=np.hstack((s1,fire_states))

  #action=chose_action(state,epsilon)